<a href="https://colab.research.google.com/github/umbcfloodbot/FloodBot/blob/main/DataAnnotate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Instal and Import Packages

In [ ]:
!pip install pydub

In [ ]:
! pip install ffmpeg-python

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
import platform
import sqlalchemy
#import mysql.connector
import requests
import json
from pandas.io.json import json_normalize
import time
import librosa
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import warnings
warnings.filterwarnings("ignore")
from pydub import AudioSegment
from moviepy.editor import *
import subprocess
import os
import sys
#import Pillow
import ffmpeg
import sys
import json
from pprint import pprint # for printing Python dictionaries in a human-readable way

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2662400/45929032 bytes (5.8%)5431296/45929032 bytes (11.8%)8626176/45929032 bytes (18.8%)11370496/45929032 bytes (24.8%)14622720/45929032 bytes (31.8%)17645568/45929032 bytes (38.4%)20668416/45929032 bytes (45.0%)23912448/45929032 bytes (52.1%)26804224/45929032 bytes (58.4%)30015488/45929032 bytes (65.4%)32874496/45929032 bytes (71.6%)35651584/45929032 bytes (77.6%)

### 2. CONNECT TO COLAB FOLDERS

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
directory = "/content/drive/My Drive/thesis_work/audio/"
weather_csv =  "/content/drive/My Drive/thesis_work/weather_csv/"
trim_directory = "/content/drive/My Drive/thesis_work/trim/"
orig_image_dir = "/content/drive/My Drive/thesis_work/orig/"
os.chdir(directory)

Mounted at /content/drive


In [ ]:
from datetime import datetime
now = datetime.now()
today_dt=now.strftime("%Y%m%d%H")

### 3. LOCATE FOLDER WITH VIDEO

In [ ]:
Vid_fldr1 = '/content/drive/My Drive/thesis_work/FlBot_Video_112721_021822'
Vid_fldr2 = '/content/drive/My Drive/thesis_work/audio/PhD_Thesis/data/video'
Vid_fldr3 = '/content/drive/My Drive/thesis_work/audio/PhD_Thesis/data/amzn_video'
Vid_fldr4 = '/content/drive/My Drive/thesis_work/audio/PhD_Thesis/data/video_2020_21'

#### 3.1 First Group of Folder

In [ ]:
import os
from glob import glob
# Iterate over the list of filepaths & remove each file.

mpg_files = [] 
mp4 = glob(Vid_fldr1+ '/*.mp4')

for j in mp4:
    try:
        #os.remove(j)
        mpg_files.append(j)
    except OSError:
        print("Error while adding file")

In [ ]:
df_vid_list1 = pd.DataFrame(mpg_files,columns = ['Orig_vid_loc'] )

In [ ]:
df_vid_list1.head(2)

,Orig_vid_loc
0,/content/drive/My Drive/thesis_work/FlBot_Video_112721_021822/Rec_20211129_142725_151_M.mp4
1,/content/drive/My Drive/thesis_work/FlBot_Video_112721_021822/Rec_20211129_142930_151_M.mp4


In [ ]:
Final_df1 = pd.DataFrame()


#for row in df_vid_list2.head(5).itertuples():
for row in df_vid_list1.itertuples():
  vid= row.Orig_vid_loc
  #print(vid)
  vid_Json1 = (ffmpeg.probe(vid)["format"])
  vid_Json2 = (ffmpeg.probe(vid)["streams"])
  with open('data.json', 'w') as f:
      json.dump(vid_Json1, f)

  with open('data1.json', 'w') as f:
      json.dump(vid_Json2, f)    

  df = pd.read_json('data.json')
  df1 = pd.read_json('data1.json')
  df_video = df1.loc[df1['codec_type'] == 'video']
  df_audio = df1.loc[df1['codec_type'] == 'audio']
  df= df[['filename','tags','duration','size']]
  df_video = df_video[['width','height','nb_frames','duration_ts','duration','avg_frame_rate']]
  df_audio = df_audio[['sample_rate','channels','nb_frames','channel_layout','max_bit_rate']]
  #df1= df1[['nb_frames','width','height']]
  #df= df[['filename','tags','duration','size']]
  time= df.iloc[1,1]
  fname= df.iloc[0,0]
  durn = df_video.iloc[0,4]
  vid_frm_rt=df_video.iloc[0,5]
  #vid_frm_ttl=df_video.iloc[0,3]
  size = df.iloc[0,3]
  wdth= df_video.iloc[0,1]
  hgth = df_video.iloc[0,0]
  vid_frames = df_video.iloc[0,2]
  aud_smpl_rt = df_audio.iloc[0,0]
  aud_frames = df_audio.iloc[0,2]
  aud_chnl = df_audio.iloc[0,3]
  aud_max_bit_rate = df_audio.iloc[0,4]
  #print(wdth)
  #print(hgth)
  #video_meta_df2 =video_meta_df1
  video_meta_df2 = pd.DataFrame([[fname,time,durn,size,wdth,hgth,vid_frames,vid_frm_rt,aud_smpl_rt,aud_frames,aud_chnl,aud_max_bit_rate]]
                                ,columns=['FileName','creation_time','Vid_lngth_sec','Vid_size_KB','Frm_Width','Frm_Height','Tot_Img_Frames',
                                          'Vid_frm_rate','Aud_sample_rate','Tot_aud_Frames','Aud_Chanel','Aud_max_bit_rate'])
  Final_df1 = pd.concat([Final_df1,video_meta_df2], ignore_index=True)

#Final_df2 =Final_df2[['FileName','creation_time']]
#Final_df2 =Final_df2[['FileName','creation_time']]
Final_df1=Final_df1.drop_duplicates()
Final_df1.to_csv('Final_df1a_'+today_dt+'.csv',index=False)
Final_df1.sample(2)

,FileName,creation_time,Vid_lngth_sec,Vid_size_KB,Frm_Width,Frm_Height,Tot_Img_Frames,Vid_frm_rate,Aud_sample_rate,Tot_aud_Frames,Aud_Chanel,Aud_max_bit_rate
1460,/content/drive/My Drive/thesis_work/FlBot_Video_112721_021822/Rec_20211127_210654_151_M.mp4,2021-11-27T21:07:10.000000Z,14.640,4922162,1072.0,1920.0,220,2750/183,16000.0,229,mono,127313.0
1479,/content/drive/My Drive/thesis_work/FlBot_Video_112721_021822/Rec_20211128_192844_151_M.mp4,2021-11-28T19:29:00.000000Z,15.241,4964087,1072.0,1920.0,229,229000/15241,16000.0,238,mono,127313.0


In [ ]:
#df1 = pd.read_csv('Final_df2022030802.csv')

In [ ]:
df1.head(2)

,index,codec_name,codec_long_name,profile,codec_type,codec_time_base,codec_tag_string,codec_tag,width,height,...,bits_per_raw_sample,nb_frames,disposition,tags,sample_fmt,sample_rate,channels,channel_layout,bits_per_sample,max_bit_rate
0,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,High,video,14707/442000,avc1,0x31637661,1920.0,1072.0,...,8.0,221,"{'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effects': 0, 'attached_pic': 0, 'timed_thumbnails': 0}","{'language': 'eng', 'handler_name': 'VideoHandler', 'encoder': 'AVC Coding'}",NaN,NaN,NaN,NaN,NaN,NaN
1,1,aac,AAC (Advanced Audio Coding),LC,audio,1/16000,mp4a,0x6134706d,NaN,NaN,...,NaN,230,"{'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effects': 0, 'attached_pic': 0, 'timed_thumbnails': 0}","{'language': 'eng', 'handler_name': 'SoundHandler'}",fltp,16000.0,1.0,mono,0.0,127313.0


#### 3.2 Second Group of Folder

In [ ]:
import os
from glob import glob
# Iterate over the list of filepaths & remove each file.

mpg_files2 = [] 
mp4 = glob(Vid_fldr2+ '/*.mp4')

for j in mp4:
    try:
        #os.remove(j)
        mpg_files2.append(j)
    except OSError:
        print("Error while adding file")

In [ ]:
df_vid_list2 = pd.DataFrame(mpg_files2,columns = ['Orig_vid_loc'] )
#print(df_vid_list2.shape)

In [ ]:
Final_df2 = pd.DataFrame()


#for row in df_vid_list2.head(5).itertuples():
for row in df_vid_list2.itertuples():
  vid= row.Orig_vid_loc
  #print(vid)
  vid_Json1 = (ffmpeg.probe(vid)["format"])
  vid_Json2 = (ffmpeg.probe(vid)["streams"])
  with open('data.json', 'w') as f:
      json.dump(vid_Json1, f)

  with open('data1.json', 'w') as f:
      json.dump(vid_Json2, f)    

  df = pd.read_json('data.json')
  df1 = pd.read_json('data1.json')
  df_video = df1.loc[df1['codec_type'] == 'video']
  df_audio = df1.loc[df1['codec_type'] == 'audio']
  df= df[['filename','tags','duration','size']]
  df_video = df_video[['width','height','nb_frames','duration_ts','duration','avg_frame_rate']]
  df_audio = df_audio[['sample_rate','channels','nb_frames','channel_layout','max_bit_rate']]
  #df1= df1[['nb_frames','width','height']]
  #df= df[['filename','tags','duration','size']]
  time= df.iloc[1,1]
  fname= df.iloc[0,0]
  durn = df_video.iloc[0,4]
  vid_frm_rt=df_video.iloc[0,5]
  #vid_frm_ttl=df_video.iloc[0,3]
  size = df.iloc[0,3]
  wdth= df_video.iloc[0,1]
  hgth = df_video.iloc[0,0]
  vid_frames = df_video.iloc[0,2]
  aud_smpl_rt = df_audio.iloc[0,0]
  aud_frames = df_audio.iloc[0,2]
  aud_chnl = df_audio.iloc[0,3]
  aud_max_bit_rate = df_audio.iloc[0,4]
  #print(wdth)
  #print(hgth)
  #video_meta_df2 =video_meta_df1
  video_meta_df2 = pd.DataFrame([[fname,time,durn,size,wdth,hgth,vid_frames,vid_frm_rt,aud_smpl_rt,aud_frames,aud_chnl,aud_max_bit_rate]]
                                ,columns=['FileName','creation_time','Vid_lngth_sec','Vid_size_KB','Frm_Width','Frm_Height','Tot_Img_Frames',
                                          'Vid_frm_rate','Aud_sample_rate','Tot_aud_Frames','Aud_Chanel','Aud_max_bit_rate'])
  Final_df2 = pd.concat([Final_df2,video_meta_df2], ignore_index=True)

#Final_df2 =Final_df2[['FileName','creation_time']]
#Final_df2 =Final_df2[['FileName','creation_time']]
Final_df2=Final_df2.drop_duplicates()
Final_df2.to_csv('Final_df2a_'+today_dt+'.csv',index=False)
Final_df2.sample(2)


,FileName,creation_time,Vid_lngth_sec,Vid_size_KB,Frm_Width,Frm_Height,Tot_Img_Frames,Vid_frm_rate,Aud_sample_rate,Tot_aud_Frames,Aud_Chanel,Aud_max_bit_rate
1502,/content/drive/My Drive/thesis_work/audio/PhD_...,2020-03-06T20:25:58.000000Z,30.025,8801560,1072.0,1920.0,451,18040/1201,16000.0,470,mono,127313.0
1317,/content/drive/My Drive/thesis_work/audio/PhD_...,2020-02-28T13:28:34.000000Z,30.092,9071662,1072.0,1920.0,452,113000/7523,16000.0,472,mono,127313.0


In [ ]:
#df1= pd.read_csv('Final_df22 02 2030811.csv')
#df2= pd.read_csv('Final_df22022030811.csv')

In [ ]:
#df2.head(2)

#### 3.3 Third Group of Folder

In [ ]:
import os
from glob import glob
# Iterate over the list of filepaths & remove each file.

mpg_files3 = [] 
mp4 = glob(Vid_fldr3+ '/*.mp4')

for j in mp4:
    try:
        #os.remove(j)
        mpg_files3.append(j)
    except OSError:
        print("Error while adding file")

In [ ]:
df_vid_list3 = pd.DataFrame(mpg_files3,columns = ['Orig_vid_loc'] )
print(df_vid_list3.shape)

(193, 1)


In [ ]:
Final_df3 = pd.DataFrame()


#for row in df_vid_list2.head(5).itertuples():
for row in df_vid_list3.itertuples():
  vid= row.Orig_vid_loc
  #print(vid)
  vid_Json1 = (ffmpeg.probe(vid)["format"])
  vid_Json2 = (ffmpeg.probe(vid)["streams"])
  with open('data.json', 'w') as f:
      json.dump(vid_Json1, f)

  with open('data1.json', 'w') as f:
      json.dump(vid_Json2, f)    

  df = pd.read_json('data.json')
  df1 = pd.read_json('data1.json')
  df_video = df1.loc[df1['codec_type'] == 'video']
  df_audio = df1.loc[df1['codec_type'] == 'audio']
  df= df[['filename','tags','duration','size']]
  df_video = df_video[['width','height','nb_frames','duration_ts','duration','avg_frame_rate']]
  df_audio = df_audio[['sample_rate','channels','nb_frames','channel_layout','max_bit_rate']]
  #df1= df1[['nb_frames','width','height']]
  #df= df[['filename','tags','duration','size']]
  time= df.iloc[1,1]
  fname= df.iloc[0,0]
  durn = df_video.iloc[0,4]
  vid_frm_rt=df_video.iloc[0,5]
  #vid_frm_ttl=df_video.iloc[0,3]
  size = df.iloc[0,3]
  wdth= df_video.iloc[0,1]
  hgth = df_video.iloc[0,0]
  vid_frames = df_video.iloc[0,2]
  aud_smpl_rt = df_audio.iloc[0,0]
  aud_frames = df_audio.iloc[0,2]
  aud_chnl = df_audio.iloc[0,3]
  aud_max_bit_rate = df_audio.iloc[0,4]
  #print(wdth)
  #print(hgth)
  #video_meta_df2 =video_meta_df1
  video_meta_df2 = pd.DataFrame([[fname,time,durn,size,wdth,hgth,vid_frames,vid_frm_rt,aud_smpl_rt,aud_frames,aud_chnl,aud_max_bit_rate]]
                                ,columns=['FileName','creation_time','Vid_lngth_sec','Vid_size_KB','Frm_Width','Frm_Height','Tot_Img_Frames',
                                          'Vid_frm_rate','Aud_sample_rate','Tot_aud_Frames','Aud_Chanel','Aud_max_bit_rate'])
  Final_df3 = pd.concat([Final_df3,video_meta_df2], ignore_index=True)

#Final_df2 =Final_df2[['FileName','creation_time']]
#Final_df2 =Final_df2[['FileName','creation_time']]
Final_df3=Final_df3.drop_duplicates()
Final_df3.to_csv('Final_df3a_'+today_dt+'.csv',index=False)
Final_df3.sample(2)

,FileName,creation_time,Vid_lngth_sec,Vid_size_KB,Frm_Width,Frm_Height,Tot_Img_Frames,Vid_frm_rate,Aud_sample_rate,Tot_aud_Frames,Aud_Chanel,Aud_max_bit_rate
105,/content/drive/My Drive/thesis_work/audio/PhD_...,2020-12-04T14:34:21.000000Z,18.244,2297926,352.0,640.0,92,23000/4561,16000.0,285,mono,127313.0
147,/content/drive/My Drive/thesis_work/audio/PhD_...,2020-11-11T21:59:27.000000Z,6.032,2050323,352.0,640.0,31,3875/754,16000.0,94,mono,127313.0


In [ ]:
#print(Final_df3.shape)

In [ ]:
#df3 = pd.read_csv('Final_df32022030811.csv')

In [ ]:
#df3.head(2)

#### 3.4 Fourth Group of Data

In [ ]:
import os
from glob import glob
# Iterate over the list of filepaths & remove each file.

mpg4_files = [] 
mp4 = glob(Vid_fldr4+ '/*.mp4')

for j in mp4:
    try:
        #os.remove(j)
        mpg4_files.append(j)
    except OSError:
        print("Error while adding file")

In [ ]:
df_vid_list4 = pd.DataFrame(mpg4_files,columns = ['Orig_vid_loc'] )
print(df_vid_list4.shape)

(1251, 1)


In [ ]:
Final_df4 = pd.DataFrame()


#for row in df_vid_list2.head(5).itertuples():
for row in df_vid_list3.itertuples():
  vid= row.Orig_vid_loc
  #print(vid)
  vid_Json1 = (ffmpeg.probe(vid)["format"])
  vid_Json2 = (ffmpeg.probe(vid)["streams"])
  with open('data.json', 'w') as f:
      json.dump(vid_Json1, f)

  with open('data1.json', 'w') as f:
      json.dump(vid_Json2, f)    

  df = pd.read_json('data.json')
  df1 = pd.read_json('data1.json')
  df_video = df1.loc[df1['codec_type'] == 'video']
  df_audio = df1.loc[df1['codec_type'] == 'audio']
  df= df[['filename','tags','duration','size']]
  df_video = df_video[['width','height','nb_frames','duration_ts','duration','avg_frame_rate']]
  df_audio = df_audio[['sample_rate','channels','nb_frames','channel_layout','max_bit_rate']]
  #df1= df1[['nb_frames','width','height']]
  #df= df[['filename','tags','duration','size']]
  time= df.iloc[1,1]
  fname= df.iloc[0,0]
  durn = df_video.iloc[0,4]
  vid_frm_rt=df_video.iloc[0,5]
  #vid_frm_ttl=df_video.iloc[0,3]
  size = df.iloc[0,3]
  wdth= df_video.iloc[0,1]
  hgth = df_video.iloc[0,0]
  vid_frames = df_video.iloc[0,2]
  aud_smpl_rt = df_audio.iloc[0,0]
  aud_frames = df_audio.iloc[0,2]
  aud_chnl = df_audio.iloc[0,3]
  aud_max_bit_rate = df_audio.iloc[0,4]
  #print(wdth)
  #print(hgth)
  #video_meta_df2 =video_meta_df1
  video_meta_df2 = pd.DataFrame([[fname,time,durn,size,wdth,hgth,vid_frames,vid_frm_rt,aud_smpl_rt,aud_frames,aud_chnl,aud_max_bit_rate]]
                                ,columns=['FileName','creation_time','Vid_lngth_sec','Vid_size_KB','Frm_Width','Frm_Height','Tot_Img_Frames',
                                          'Vid_frm_rate','Aud_sample_rate','Tot_aud_Frames','Aud_Chanel','Aud_max_bit_rate'])
  Final_df4 = pd.concat([Final_df4,video_meta_df2], ignore_index=True)

#Final_df2 =Final_df2[['FileName','creation_time']]
#Final_df2 =Final_df2[['FileName','creation_time']]
Final_df4=Final_df4.drop_duplicates()
Final_df4.to_csv('Final_df4a_'+today_dt+'.csv',index=False)
Final_df4.sample(2)

In [ ]:
#df4 = pd.read_csv('Final_df4_2022030910.csv')

In [ ]:
#df4.head(3)

### 4. Annotate all Videos with 5 Minute TimeStamps

In [ ]:
Final_df = pd.concat([Final_df1,Final_df2,Final_df3,Final_df4], ignore_index=True)

In [ ]:
All_video_df.shape

In [ ]:
#Final_df =All_video_df[['FileName','creation_time']]

In [ ]:
#Final_df = All_video_df

In [ ]:
#Final_df1 =Final_df1[['FileName','creation_time']]
Final_df=Final_df.drop_duplicates()
Final_df.to_csv('Final_dfa_'+today_dt+'.csv',index=False)

In [ ]:
#Final_df=Final_df.drop_duplicates()

In [ ]:
#Final_df.head(2)

In [ ]:
vid_strftime = "%Y-%m-%dT%H:%M"

In [ ]:
vid_strftime = "%Y-%m-%dT%H:%M"
Final_df['Date']=Final_df['creation_time'].str[:-11]
Final_df['Date'] = pd.to_datetime(Final_df['Date'],format=vid_strftime)
Final_df['Date']=Final_df['Date'].dt.tz_localize('utc').dt.tz_convert('US/Eastern')
Final_df['Date'] = pd.to_datetime(Final_df['Date'].dt.tz_localize(None))
#Final_df['Date'] = Final_df['Date'].dt.tz_localize(None)

In [ ]:
#Final_df.tail(2)

In [ ]:
#sample_mp4 = '/content/drive/My Drive/thesis_work/FlBot_Video_112721_021822/Rec_20220218_211712_151_M.mp4'

In [ ]:
#from IPython.display import HTML
#from base64 import b64encode
#mp4 = open(sample_mp4,'rb').read()
#data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#HTML("""
#<video width=600 controls>
#      <source src="%s" type="video/mp4">
#</video>
#""" % data_url)

In [ ]:
Final_df['fivemin_ts'] =Final_df['Date'].dt.round('5min')
Final_df['fivemin_ts']=Final_df['fivemin_ts'].apply(lambda x: str(x))
Final_df['fivemin_ts'] =Final_df['fivemin_ts'].str[:16]

In [ ]:
#Final_df.head()

In [ ]:
df_fv_min = Final_df.groupby('fivemin_ts').apply(lambda x: x['FileName'].unique())
df_fv_min = df_fv_min.apply(pd.Series)
df_fv_min['fivemin_ts'] = df_fv_min.index
df_fv_min.reset_index(drop=True, inplace=True)
df_fv_min["time_id"] = df_fv_min.index + 1
df_vid_list = df_fv_min[['time_id','fivemin_ts',0]]
df_all_fv_min = df_fv_min[['time_id','fivemin_ts',0, 1, 2, 3, 4, 5]]
df_all_fv_min.to_csv('all_vid_fv_min_'+today_dt+'.csv',index=False)

In [ ]:
df_all_fv_min.to_csv('all_vid_fv_min_'+today_dt+'.csv',index=False)

In [ ]:
#df_all_fv_min.head(2)

In [ ]:
df_vid_list = df_vid_list.rename(columns={0: "Fv_min_src_Video"})

In [ ]:
#Final_df.head(2)

In [ ]:
df_final_vid_lst = pd.merge(Final_df, df_vid_list, how='inner', left_on = 'FileName', right_on = 'Fv_min_src_Video')

In [ ]:
#df_final_vid_lst.head()

In [ ]:
df_final_vid_lst = df_final_vid_lst[['time_id', 'FileName', 'creation_time', 'Date', 'fivemin_ts_x' ]]
df_final_vid_lst = df_final_vid_lst.rename(columns={'fivemin_ts_x': "Fivemin_Time",'Date':'Orig_Video_Date','creation_time':'UTC_creation_time'})
df_final_vid_lst.to_csv('final_vid_lst_'+today_dt+'.csv',index=False)

In [ ]:
#df_final_vid_lst.head(2)

In [ ]:
#df_final_vid_lst.sort_values(by=['time_id']).head(2)


In [ ]:
#df_final_vid_lst.sort_values(by=['time_id']).tail(5)

In [ ]:
df_final_vid_lst.to_csv('df_final_vid_lst_'  + today_dt+'.csv',index=False)

In [ ]:
#sample_mp4='/content/drive/My Drive/thesis_work/audio/PhD_Thesis/data/video/Rec_20200328_174707_151_M.mp4'

In [ ]:
#from IPython.display import HTML
#from base64 import b64encode
#mp4 = open(sample_mp4,'rb').read()
#data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#HTML("""
#<video width=600 controls>
#      <source src="%s" type="video/mp4">
#</video>
#""" % data_url)

In [ ]:
from datetime import date,datetime,timedelta

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
#df_final_vid_lst.head(2)

In [ ]:
df_final_vid_lst['Time_Index'] = df_final_vid_lst['Orig_Video_Date']

In [ ]:
df_final_vid_lst['Day']=df_final_vid_lst['Orig_Video_Date'].apply(lambda x: str(x))
df_final_vid_lst['Day'] =df_final_vid_lst['Day'].str[:10]
df_final_vid_lst['Month']=df_final_vid_lst['Orig_Video_Date'].apply(lambda x: str(x))
df_final_vid_lst['Month'] =df_final_vid_lst['Day'].str[:7]

In [ ]:
df_mnthly_video = df_final_vid_lst.groupby('Month').size()

In [ ]:
#type(df_mnthly_video)

In [ ]:
#df_monthly_video = pd.DataFrame(df_mnthly_video)

In [ ]:
#df_monthly_video

In [ ]:
#df_final_vid_lst['Month'] = df_monthly.index
#df_final_vid_lst['Month'] = pd.to_datetime(df_final_vid_lst['Orig_Video_Date'], format='%y%m%d')
#df_monthly.reset_index(level=0, inplace=True)
#df_final_vid_lst['Month'] = pd.DatetimeIndex(df_final_vid_lst['Orig_Video_Date']).month

In [ ]:
#df_monthly_video = df_final_vid_lst.groupby('Time_Index', as_index = False)['Month'].count()

In [ ]:
#df_monthly_video

In [ ]:
#df_daily_video.head(2)

In [ ]:
#df_final_vid_lst['Month'] = df_daily_video.Orig_Video_Date.apply(lambda x: x.strftime('%Y%m%d')).astype(int)

In [ ]:
#df_final_vid_lst.set_index('Time_Index', inplace=True)

In [ ]:
#df_final_vid_lst.head()
#df_video_monthly = df_final_vid_lst.resample('M').sum()

In [ ]:
#df_video_monthly.head(2)

### 5. Weather Data

In [ ]:
import glob
path = r'/content/drive/My Drive/thesis_work/audio/PhD_Thesis/data/weather_data'
all_files = glob.glob(path + "/*.txt")
df_files = (pd.read_csv(f) for f in all_files)
df_weather   = pd.concat(df_files, ignore_index=True)

In [ ]:
df_weather.head(2)